# Data Ingestion and Data Wrangling 

Capstone 10: Agronomics 

In [1]:
import os
import pandas as pd
import csv
import numpy as np
from functools import reduce
from pandas import DataFrame, Series 
import glob

Combining crop data for 1980 to 2015 into one dataframe. The original crop data files were initially named to have the same file ending "cropyear_production.csv"

In [ ]:
# Taking ALL the csv files for crop production. This was manually put in one folder by project creators 

path =r'csv_data/'
filenames = glob.glob(path + "/*cropyear_production.csv")

#Creates an empty list for a dataframe (sets up data frame for combining all crop data files into one dataframe)
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
    
# Concatenate all data into one DataFrame (within the empty list)
big_frame = pd.concat(dfs, ignore_index=True)

In [ ]:
# Create a csv file named CROP_PRODUCTION_ALL for the data from big_frame
with open('CROP_PRODUCTION_ALL.csv', 'w') as f:
    big_frame.to_csv(f, header=False,index=0)

In [ ]:
# Create column names 
colnames=['Year','Commodity_Code','Crop_Name','County_Code','County','Harvested','Yield','Production','Price','Unit','Value'] 

# Adds the column names to the dataframe 
crop_production=pd.read_csv('CROP_PRODUCTION_ALL.csv', names=colnames, header=None,dtype=object)

Creating files for California counties of interest: 
1. Fresno County 
2. Kern County 

In [ ]:
# Create new data frame for Fresno County 

crop_production_Fresno=crop_production[crop_production['County'].dropna().str.contains("Fresno") & crop_production['Crop_Name'].dropna().str.contains("ALMONDS ALL")]

#Makes all counties named Fresno to remove manual error (from original dataset). This makes the data consistent. 
crop_production_Fresno['County']='Fresno'

#crop_production_Fresno.loc[:,'County']='Fresno'

In [ ]:
#Create new data frame for Kern County 

crop_production_Kern=crop_production[crop_production['County'].dropna().str.contains("Kern") & crop_production['Crop_Name'].dropna().str.contains("ALMONDS ALL")]

#Makes all counties named Kern to remove manual error ( from original dataset). This makes the data consistent. 
crop_production_Kern['County']='Kern'

#crop_production_Kern.loc[:,'County']='Kern'

In [ ]:
# Confirm dataframe 
with open('1-CROP_PRODUCTION-Fresno.csv', 'w') as f:
    
    crop_production_Fresno.to_csv(f, header=True,index=0)
f.close()

In [ ]:
# Confirm dataframe 
with open('1-CROP_PRODUCTION-Kern.csv', 'w') as f:
    
    crop_production_Kern.to_csv(f, header=True,index=0)
f.close()

In [ ]:
# Define columns for analysis 

f_crop_f=pd.read_csv('1-CROP_PRODUCTION-Fresno.csv',usecols=['Year','Commodity_Code','County_Code','County','Harvested','Yield','Production','Price','Value'],dtype={'Year':int,'Harvested':int,'Production':int,'Price':float})
df_crop_k=pd.read_csv('1-CROP_PRODUCTION-Kern.csv',usecols=['Year','Commodity_Code','County_Code','County','Harvested','Yield','Production','Price','Value'],dtype={'Year':int,'Harvested':int,'Production':int,'Price':float})

# Data Ingestion and Wrangling 

# Precipitation  and Temperature Data 
# INSERT LINKS TO DATA SOURCES 
Precipiation: Years 1980 to 2016
Temperature: Years 1980 to 2016 

In [ ]:
# Fresno County Annual Precipitation (inches)

df_prec_f=pd.read_csv("f_temp_prec/Fresno_Rainfall_Yearly.csv")
df_prec_f.rename(columns={
                 'January': 'January_p',
                 'February': 'February_p',
                 'March': 'March_p',
                 'April': 'April_p',
                 'May': 'May_p',
                 'June': 'June_p',
                 'July': 'July_p',
                 'August': 'August_p',
                 'September': 'September_p',
                 'October': 'October_p',
                 'November': 'November_p',
                 'December': 'December_p',
                 'Total': 'Total_p'}, inplace=True)

In [ ]:
#The precipitation data includes values set as 'T' for Trace. 
#Trace are precipitation measurments where precipitation occured but did not measure atleast 0.01 inches. 
# The code below gives a random number between 0.001 and 0.005 for Trace values 

df_prec_f.replace(['T'],np.random.uniform(+0.001,+0.005), inplace=True)
df_prec1_f=df_prec_f[(df_prec_f['Year']>='1980') & (df_prec_f['Year']<'2016')]
df_prec1_f = df_prec1_f.astype(float)

In [ ]:
with open('2-RAINFALL_FRESNO.csv', 'w') as f:
    df_prec1_f.to_csv(f, header=True,index=0,float_format='%.3f') #take 3 decimals places 
f.close()

In [ ]:
df_prec_k=pd.read_csv("k_temp_prec/Kern_Rainfall_Yearly.csv")
df_prec_k.columns=['Year','January_p','February_p','March_p','April_p','May_p','June_p','July_p','August_p','September_p','October_p','November_p','December_p','Total_p']

In [ ]:
df_prec_k.replace(['T'],np.random.uniform(+0.001,+0.005), inplace=True)
df_prec_k = df_prec_k.astype(float)
# data needs to be cleaned and consistent 
df_prec_k.replace(r'[\-]+', r'', regex=True, inplace=True)
df_prec1_k=df_prec_k[(df_prec_k['Year']>=1980) & (df_prec_k['Year']<2016)]
df_prec1_k = df_prec_k.astype(float)

In [ ]:
with open('2-RAINFALL_KERN.csv', 'w') as f:
    #df_prec1_k = df_prec1_k.astype(float)
    df_prec1_k.to_csv(f, header=True,index=0,float_format='%.3f')
f.close()

In [ ]:
# Temperature (degrees Fahrenhiet)
df_temp_f=pd.read_csv("f_temp_prec/Fresno_Average_Monthly_Temperatures.csv")
df_temp_f.rename(columns={
                 'January': 'January_t',
                 'February': 'February_t',
                 'March': 'March_t',
                 'April': 'April_t',
                 'May': 'May_t',
                 'June': 'June_t',
                 'July': 'July_t',
                 'August': 'August_t',
                 'September': 'September_t',
                 'October': 'October_t',
                 'November': 'November_t',
                 'December': 'December_t',
                 'Annual': 'Annual_t'}, inplace=True)

In [ ]:
df_temp1_f=df_temp_f[(df_temp_f['Year']>=1980) & (df_temp_f['Year']<2016)]
df_temp1_f = df_temp1_f.astype(float)

In [ ]:
df_temp_k=pd.read_csv("k_temp_prec/Kern_Average_Temperatures.csv")
df_temp_k.rename(columns={
                 'January': 'January_t',
                 'February': 'February_t',
                 'March': 'March_t',
                 'April': 'April_t',
                 'May': 'May_t',
                 'June': 'June_t',
                 'July': 'July_t',
                 'August': 'August_t',
                 'September': 'September_t',
                 'October': 'October_t',
                 'November': 'November_t',
                 'December': 'December_t',
                 'Annual': 'Annual_t'}, inplace=True)

In [ ]:
df_temp_k.columns=['Year','January_t','February_t','March_t','April_t','May_t','June_t','July_t','August_t','September_t','October_t','November_t','December_t','Annual_t']
df_temp_k.replace(r'[\-]+', r'', regex=True, inplace=True)
df_temp1_k=df_temp_k[(df_temp_k['Year']>=1980) & (df_temp_k['Year']<2016)]

# Census Data 
# Insert data source links 
Year: 1980 - 2015

In [ ]:
Fresno County Census 

In [ ]:
f_census1=pd.read_csv("census_data/fresno_percapita_personal_income.csv")
f_census2=pd.read_csv("census_data/fresno_personal_income.csv")
f_census3=pd.read_csv("census_data/fresno-resident_population.csv")
f_census4=pd.read_csv("census_data/fresno_house_price_index.csv")

In [ ]:
f_census_df=[f_census1,f_census2,f_census3,f_census4]
fresno_census_concat = reduce(lambda  left,right: pd.merge(left,right,on='DATE', how='left'), f_census_df)
fresno_census_concat['DATE'] = fresno_census_concat['DATE'].map(lambda x: str(x)[:-6])
with open('3-FRESNO_CENSUS_DATA.csv', 'w') as f:
    fresno_census_concat.to_csv(f, header=True,index=0)

col_census=['Year','Percapita_Personal_Income','Personal_Income','Resident_Population','House_Price_Index']
fresno_census=pd.read_csv('FRESNO_CENSUS_DATA.csv', names=col_census)
fresno_census=fresno_census[(fresno_census['Year']>='1980') & (fresno_census['Year']<'2016')]
fresno_census = fresno_census.astype(float)

Kern County Census 

In [ ]:
k_census1=pd.read_csv("census_data/kern_percapita_personal_income.csv")
k_census2=pd.read_csv("census_data/kern_personal_income.csv")
k_census3=pd.read_csv("census_data/kern_resident_population.csv")
k_census4=pd.read_csv("census_data/kern_house_price_index.csv")

In [ ]:
# Merge all four census data files into one file 

k_census_df=[k_census1,k_census2,k_census3,k_census4]
kern_census_concat = reduce(lambda  left,right: pd.merge(left,right,on='DATE', how='left'), k_census_df)
kern_census_concat['DATE'] = kern_census_concat['DATE'].map(lambda x: str(x)[:-6])

kern_census_concat.rename(columns={
                 'DATE': 'Year',
                 'PCPI06029': 'Percapita_Personal_Income',
                 'PI06029': 'Personal_Income',
                 'CAKERN0POP': 'Resident_Population',
                 'ATNHPIUS06029A': 'House_Price_Index'}, inplace=True)

In [ ]:
with open('3-KERN_CENSUS_DATA.csv', 'w') as f:
    kern_census_concat.to_csv(f, header=True,index=0)

In [ ]:
kern_census=pd.read_csv('KERN_CENSUS_DATA.csv')

In [ ]:
kern_census1=kern_census[(kern_census['Year']>=1980) & (kern_census['Year']<2016)]

In [ ]:
with open('4-TEMPERATURES_FRESNO.csv', 'w') as f:
    df_temp1_k = df_temp1_k.astype(float)

    df_temp1_f.to_csv(f, header=True,index=0)
f.close()

In [ ]:
with open('4-TEMPERATURES_KERN.csv', 'w') as f:
    df_temp1_k.to_csv(f, header=True,index=0)
f.close()

# Combine Crop Production, Weather, and Census Data 

Fresno County Combined File 

In [ ]:
fresno_data_frames=[df_crop_f,df_prec1_f,df_temp1_f,fresno_census]

df_concat_fresno = reduce(lambda  left,right: pd.merge(left,right,on='Year', how='left'), fresno_data_frames)

In [ ]:
Kern County Combined File 

In [ ]:
kern_data_frames=[df_crop_k,df_prec1_k,df_temp1_k,kern_census]

df_concat_kern = reduce(lambda  left,right: pd.merge(left,right,on='Year', how='left'), kern_data_frames)

In [ ]:
#Merged all csv files: Crop, weather, and censu
with open('5-MERGE.csv', 'w') as f:
    df_concat_fresno.to_csv(f, header=True,index=0)
    df_concat_kern.to_csv(f, header=False,index=0)
f.close()

# Almond Production Growing Season 

Defined growing seasons and added as new columns to 5-Merge dataframe. 
Precipiation is sum of growing season months (Feburary - June)
Temperature is the adverage temperature for growing season months (Feburary- June)

In [ ]:
# Added new columns based on the months for growing seasons

merge_almond_production=pd.read_csv("5-MERGE.csv")
merge_almond_production.insert(9, 'Grow_total_p', (merge_almond_production['February_p']+merge_almond_production['March_p']+merge_almond_production['April_p']+merge_almond_production['May_p']+merge_almond_production['June_p']))
merge_almond_production.insert(10, 'Grow_avg_t', (merge_almond_production['February_t']+merge_almond_production['March_t']+merge_almond_production['April_t']+merge_almond_production['May_t']+merge_almond_production['June_t'])/5)

In [ ]:
with open('5-MERGE.csv', 'w') as f:
    merge_almond_production.to_csv(f, header=True,index=0)

f.close()
print(merge_almond_production.shape)